In [1]:
import keras
import tensorflow as tf
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Flatten, Conv2D, MaxPooling2D

2023-05-08 18:22:18.560720: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
(X_train, y_train), (X_valid, y_valid) = mnist.load_data()

In [3]:
X_train.shape, X_valid.shape

((60000, 28, 28), (10000, 28, 28))

In [4]:
# y = labels
y_train.shape, y_valid.shape

((60000,), (10000,))

In [5]:
X_train = X_train.reshape(60000, 28, 28, 1)
X_valid = X_valid.reshape(10000, 28, 28, 1)

X_train.shape, X_valid.shape

((60000, 28, 28, 1), (10000, 28, 28, 1))

In [6]:
# for better imagination of shape, try to find last values in every index
X_train[59999][27][27][0]
# This seems very simple but in this approach you can find seed to imagine this shape, 
# and get intuition how to get access to every places in this space.

0

In [7]:
# do you see differences? 
X_train.dtype, X_train.astype('float32').dtype

(dtype('uint8'), dtype('float32'))

In [8]:
X_train = X_train.astype('float32')
X_valid = X_valid.astype('float32')

X_train.dtype, X_valid.dtype

(dtype('float32'), dtype('float32'))

In [9]:
X_train[0, 0:27, 0:27, 0]

array([[  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.],
       [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0

In [10]:
t = tf.constant([[10, 10, 2, 2]])
t[0][0:4]
# w tym momencie coś mi się pojebało... tak dosłownie . .

2023-05-08 18:22:21.958447: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-08 18:22:21.959899: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


<tf.Tensor: shape=(4,), dtype=int32, numpy=array([10, 10,  2,  2], dtype=int32)>

In [11]:
# this replace current numbers (from range 0 255) to values from range between 0 and 1
X_train /= 255
X_valid /= 255

In [12]:
len(y_train), len(y_valid)

(60000, 10000)

In [13]:
y_train[0:2], y_valid[0:2]

(array([5, 0], dtype=uint8), array([7, 2], dtype=uint8))

In [14]:
# look differences between above and under y_train
num_classes = 10
y_train = tf.keras.utils.to_categorical(y_train, num_classes=num_classes)
y_valid = tf.keras.utils.to_categorical(y_valid, num_classes=num_classes)

y_train[0:2], y_valid[0:2]

(array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32),
 array([[0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32))

# CREATE MODEL ARCHITECTURE

In [15]:
# create empty model
model = Sequential()

In [16]:
type(model)

keras.engine.sequential.Sequential

In [17]:
# First convolution layer
model.add(Conv2D(32, 
                 kernel_size=(3, 3), 
                 activation='relu', 
                 input_shape=(28, 28, 1)
                 ))

In [18]:
# Second convolution layer
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(2, 2))
tf.random.set_seed(0)
model.add(Dropout(0.25))
model.add(Flatten())

In [19]:
# Third hidden layer
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))


In [20]:
# Output layer
model.add(Dense(num_classes, activation='softmax'))

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 64)        18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 12, 12, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 12, 12, 64)        0         
                                                                 
 flatten (Flatten)           (None, 9216)              0         
                                                                 
 dense (Dense)               (None, 128)               1179776   
                                                        